# Exercise 7 - Sequence to Sequence Learning for Addition of Two Numbers
In this example, we train a model to learn to add two numbers, provided as strings.
Example:

-    Input: "512+23"
-    Output: "535"

Input may optionally be reversed, which was shown to increase performance. Theoretically, sequence order inversion introduces shorter term dependencies between source and target for this problem.

In [1]:
#importing dependencies
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '323+562'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS


## Data Generation

In [2]:
class CharacterTable:
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """

    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[x] for x in x)


# All the numbers, plus sign and space for padding.
chars = "0123456789+ "
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print("Generating data...")
while len(questions) < TRAINING_SIZE:
    f = lambda: int(
        "".join(
            np.random.choice(list("0123456789"))
            for i in range(np.random.randint(1, DIGITS + 1))
        )
    )
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = "{}+{}".format(a, b)
    query = q + " " * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += " " * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print("Total questions:", len(questions))


Generating data...
Total questions: 50000


## Vectorize Data

In [3]:
print("Vectorization...")
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print("Training Data:")
print(x_train.shape)
print(y_train.shape)

print("Validation Data:")
print(x_val.shape)
print(y_val.shape)



Vectorization...
Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


## Building the Model

In [4]:
print("Build model...")
num_layers = 1  # Try to add more LSTM layers!

model = keras.Sequential()
# "Encode" the input sequence using a LSTM, producing an output of size 128.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(layers.LSTM(128, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(num_layers):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(layers.LSTM(128, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()


Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               72192     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 4, 128)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
dense (Dense)                (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


## Training the Model

In [5]:
epochs = 30
batch_size = 32


# Train the model each generation and show predictions against the validation
# dataset.
for epoch in range(1, epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)



Iteration 1
1407/1407 [==============================] - 33s 24ms/step - loss: 1.7613 - accuracy: 0.3549 - val_loss: 1.5918 - val_accuracy: 0.4069
Q 852+374 T 1226 ☒ 1222
Q 286+96  T 382  ☒ 282 
Q 136+472 T 608  ☒ 778 
Q 655+172 T 827  ☒ 808 
Q 637+321 T 958  ☒ 808 
Q 776+82  T 858  ☒ 872 
Q 77+160  T 237  ☒ 278 
Q 492+4   T 496  ☒ 44  
Q 54+88   T 142  ☒ 141 
Q 92+547  T 639  ☒ 902 

Iteration 2
1407/1407 [==============================] - 34s 24ms/step - loss: 1.3821 - accuracy: 0.4825 - val_loss: 1.2097 - val_accuracy: 0.5493
Q 827+596 T 1423 ☒ 1425
Q 789+65  T 854  ☒ 861 
Q 382+8   T 390  ☑ 390 
Q 0+657   T 657  ☒ 665 
Q 471+315 T 786  ☒ 765 
Q 4+942   T 946  ☒ 952 
Q 13+145  T 158  ☒ 155 
Q 471+561 T 1032 ☒ 905 
Q 216+230 T 446  ☒ 462 
Q 604+817 T 1421 ☒ 1411

Iteration 3
1407/1407 [==============================] - 35s 25ms/step - loss: 1.0680 - accuracy: 0.6033 - val_loss: 0.9771 - val_accuracy: 0.6329
Q 349+112 T 461  ☒ 457 
Q 853+23  T 876  ☒ 885 
Q 889+799 T 1688 ☒ 1755
Q 18

1407/1407 [==============================] - 32s 23ms/step - loss: 0.0181 - accuracy: 0.9955 - val_loss: 0.0077 - val_accuracy: 0.9984
Q 275+19  T 294  ☑ 294 
Q 85+998  T 1083 ☑ 1083
Q 879+5   T 884  ☑ 884 
Q 66+507  T 573  ☑ 573 
Q 1+729   T 730  ☑ 730 
Q 7+310   T 317  ☑ 317 
Q 491+995 T 1486 ☑ 1486
Q 59+515  T 574  ☑ 574 
Q 469+11  T 480  ☑ 480 
Q 586+694 T 1280 ☑ 1280

Iteration 23
1407/1407 [==============================] - 32s 23ms/step - loss: 0.0305 - accuracy: 0.9910 - val_loss: 0.0134 - val_accuracy: 0.9966
Q 657+411 T 1068 ☑ 1068
Q 395+505 T 900  ☑ 900 
Q 104+98  T 202  ☑ 202 
Q 928+473 T 1401 ☑ 1401
Q 622+0   T 622  ☑ 622 
Q 607+191 T 798  ☒ 898 
Q 325+546 T 871  ☑ 871 
Q 90+59   T 149  ☑ 149 
Q 92+814  T 906  ☑ 906 
Q 642+679 T 1321 ☑ 1321

Iteration 24
1407/1407 [==============================] - 32s 23ms/step - loss: 0.0157 - accuracy: 0.9958 - val_loss: 0.0074 - val_accuracy: 0.9987
Q 74+936  T 1010 ☑ 1010
Q 920+526 T 1446 ☑ 1446
Q 88+847  T 935  ☑ 935 
Q 893+901 T 179